In [ ]:
from aux import *

In [ ]:
# The sequence of protein to fold
# prot_seq = "YYDPETGTWY"
prot_seq = "YYDPET"

In [ ]:
prot_seq, q, energy_expr, ene_terms_expr = prepare_quantum(prot_seq, return_Hs=True)

In [ ]:
energy_func = lambdify(q, energy_expr)

In [ ]:
sched = np.power(2, np.linspace(4, -1, 300))
max(sched), min(sched), len(sched)

In [ ]:
plt.plot(sched)
plt.show()

In [ ]:
plt.plot(np.exp(-10/sched))
plt.show()

In [ ]:
num_cores = 12

res = Parallel(n_jobs=num_cores)(delayed(simulate_quantum)(prot_seq, q, energy_expr, sched, jobid) for jobid in range(720))

In [ ]:
tra, H, E, seeds, results = zip(*res)

In [ ]:
tra = np.array(tra)
history = np.array(H)
energy = np.array(E)
results = np.array(results)

In [ ]:
acp = history.cumsum(axis=1)

In [ ]:
plt.plot(acp.mean(axis=0))
plt.fill_between(
    range(acp.shape[1]),
    acp.mean(axis=0) - acp.std(axis=0),
    acp.mean(axis=0) + acp.std(axis=0), alpha=0.2)
plt.show()

In [ ]:
enes = np.array([energy_func(*qs) for qs in results])
best = np.argmin(enes)

In [ ]:
plt.hist(enes)
plt.show()

In [ ]:
correct = results[enes.round(4) == -6.20]

In [ ]:
make_viz(tra[best], prot_seq, schedule=sched, movie=False, energy_func=energy_func)

In [ ]:
if len(correct) > 0:
    make_viz(correct[[0]], prot_seq, schedule=sched, movie=False, energy_func=energy_func)

In [ ]:
all_best =enes == min(enes)
(all_best).sum()

In [ ]:
all_correct = enes.round(2) == -6.20
(all_correct).sum()

In [ ]:
plt.plot(np.sort(enes))
plt.show()

In [ ]:
rnd_best = np.random.choice(np.argwhere(all_best).reshape((-1,)))
print(rnd_best)
plt.plot(energy[rnd_best])
plt.ylim(min(enes)-5, 200)
plt.show()

## Now Using DWave to find solution

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite, FixedEmbeddingComposite
import dwave_networkx as dnx
from minorminer import find_embedding
import dimod
from sympy.parsing.sympy_parser import parse_expr

import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Define the solver and get its correspondiing adjacency graph for embedding
# other sampler parameters are in dwave config file
solver = DWaveSampler(solver="DW_2000Q_5")
solver_G = nx.Graph(solver.edgelist)

In [ ]:
print("Maximum anneal schedule points: {}".format(solver.properties["max_anneal_schedule_points"]))

In [ ]:
annealing_range = solver.properties["annealing_time_range"]
max_slope = 1.0/annealing_range[0]
print("Annealing time range: {}".format(solver.properties["annealing_time_range"]))
print("Maximum slope:", max_slope)

In [ ]:
# fix first 3 qbits to avoid redundant conformations
# for multiple problems in same graph
energy_expr_less_00 = preprocess_expr(energy_expr.subs({'q0000':0, 'q0001':1, 'q0002':0}), q)
energy_expr_less_00_func = lambdify(q[3:], energy_expr_less_00)
energy_expr_ot_00 = energy_expr_less_00.as_ordered_terms()

In [ ]:
hubo_00 = {}
for coeff, qqss in [term.as_coeff_mul() for term in energy_expr_ot_00]:
    try:
        hubo_00[qqss[1:]] = float(qqss[0]*coeff)
    except TypeError:
        hubo_00[qqss] = float(coeff)

In [ ]:
poly_00 = dimod.BinaryPolynomial(hubo_00, dimod.BINARY)
poly_00.normalize()

In [ ]:
mk_quaq_strength = 1.05

In [ ]:
bqm_00 = dimod.make_quadratic(poly_00, mk_quaq_strength, dimod.BINARY)
bqm_00.normalize()

In [ ]:
bqm_spin_00 = bqm_00.change_vartype(dimod.SPIN, inplace=False)
bqm_spin_00.normalize()

Will setup several copies of the problem in a single embedding, this way several solutions are retrieved from a single run

In [ ]:
def create_subproblem(i):
    qi = symbols([f'q{i:02d}{j:04d}' for j in range(len(q))])
    energy_expr_less = energy_expr_less_00.subs(dict(zip(q, qi)))
    return qi, energy_expr_less

In [ ]:
# Nduplicates = 170 # good for setup with 4 residues
Nduplicates = 7 # good for setup with 6 residues

In [ ]:
num_cores = 12
par_res = Parallel(n_jobs=num_cores)(delayed(create_subproblem)(i) for i in range(Nduplicates))

In [ ]:
qii, energy_expr_less = zip(*par_res)
qii = dict(zip(range(len(qii)), qii))
energy_expr_less = sum(energy_expr_less)
energy_expr_less = energy_expr_less.evalf()
energy_expr_ot = energy_expr_less.as_ordered_terms()

hubo = {}
for coeff, qqss in [term.as_coeff_mul() for term in energy_expr_ot]:
    try:
        hubo[qqss[1:]] = float(qqss[0]*coeff)
    except TypeError:
        hubo[qqss] = float(coeff)

poly = dimod.BinaryPolynomial(hubo, dimod.BINARY)
poly.normalize()
bqm = dimod.make_quadratic(poly, mk_quaq_strength, dimod.BINARY)
bqm.normalize()

bqm_spin = bqm.change_vartype(dimod.SPIN, inplace=False)
bqm_spin.normalize()

# Now make sure an embedding can be found
problem = nx.Graph()
for k, v in dict(bqm.linear).items():
    problem.add_node(k)
for k, v in dict(bqm.quadratic).items():
    problem.add_edge(*k)

In [ ]:
num_cores = 12
embedding_list = Parallel(n_jobs=num_cores)(delayed(find_embedding)(problem, solver_G) for jobid in range(num_cores))

In [ ]:
assigned_qbits = np.array([len(sum(emb.values(), [])) if len(emb) > 0 else np.nan for emb in embedding_list])
assert bool(sum(assigned_qbits[~np.isnan(assigned_qbits)])), "No embedding has been found"
print(np.sort(assigned_qbits[~np.isnan(assigned_qbits)]))

embedding = embedding_list[np.nanargmin(assigned_qbits)]

all_assigned_qbits = sum(embedding.values(), [])
print(f"Using {len(all_assigned_qbits)}/{solver.properties['num_qubits']} qbits")

In [ ]:
sampler = FixedEmbeddingComposite(solver, embedding=embedding)  

In [ ]:
# Find energy of optimal solution
bst_00 = {}
for j, qj in enumerate(q):
    bst_00[qj] = results[best, j]

smpl_00 = dict(zip(list(bqm_00.variables), [None]*len(bqm_00.variables)))
for key in bqm_00.variables:
    if type(key) == str:
        k1, k2 = parse_expr(key).as_coeff_mul()[1]
        smpl_00[key] = bst_00[k1] * bst_00[k2]
    else:
        smpl_00[key] = bst_00[key]

smpl_spin_00 = smpl_00.copy()
for key, val in smpl_00.items():
    smpl_spin_00[key] = 2*val -1

mene_00 = bqm_00.energy(smpl_00)
mene_spin_00 = bqm_spin_00.energy(smpl_spin_00)
print((mene_00, mene_spin_00))
# viz_short_smpl(smpl_00, prot_seq, q, energy_func)

In [ ]:
# Corroborate energy of optimal solution in tandem sample
bst = {}
for i, qi in qii.items():
    for j, qij in enumerate(qi):
        bst[qij] = results[best, j]

smpl = dict(zip(list(bqm.variables), [None]*len(bqm.variables)))
smpl_spin = smpl.copy()
for key, val in smpl.items():
    if type(key) == str:
        k1, k2 = parse_expr(key).as_coeff_mul()[1]
        smpl[key] = bst[k1] * bst[k2]
    else:
        smpl[key] = bst[key]
    smpl_spin[key] = 2 * smpl[key] - 1

mene = bqm.energy(smpl)
mene_spin = bqm_spin.energy(smpl_spin)

# after normalizations, the energy landscape has been scaled
# this is the target energy for the true minimum (as found by conventional simulation)

# should be equal to the min energy for a single problem
assert round(mene / Nduplicates - mene_00, 10) == 0, "Found a problem with tandem model energies"

# should be equal to the min energy for a single problem
assert round(mene_spin / Nduplicates - mene_spin_00, 10) == 0, "Found a problem with tandem model energies"

In [ ]:
subsdic = {}
keybook = {}
for i, qi in qii.items():
    tmp = dict(zip(qi, q))

    for key in bqm.variables:
        if key in qi:
            keybook[key] = i
            subsdic[key] = tmp[key]
        elif type(key) == str:
            k1, k2 = parse_expr(key).as_coeff_mul()[1]
            if k1 in qi:
                keybook[key] = i

In [ ]:
def separate_subsamples(smpl, occurrences=1, vartype=dimod.BINARY):

    # separate samples in batch
    samples = [{} for i in range(Nduplicates)]

    for key, val in smpl.items():
        i = keybook[key]
        if type(key) == str:
            k1, k2 = parse_expr(key).as_coeff_mul()[1]
            k1 = k1.subs(k1, subsdic[k1])
            k2 = k2.subs(k2, subsdic[k2])
            nkey = f'{k1}*{k2}'
            if nkey not in bqm_00.variables:
                nkey = f'{k2}*{k1}'
        else:
            nkey = key.subs(key, subsdic[key])

        if vartype == dimod.SPIN:
            val = int(val/2 + 0.5)

        samples[i][nkey] = val

    return samples, [occurrences] * Nduplicates

In [ ]:
def check_valid_bqsample(sample):
    valid = True
    for k, v in sample.items():
        if type(k) == str:
            k1, k2 = parse_expr(k).as_coeff_mul()[1]
            valid = valid and (sample[k1] * sample[k2] == sample[k])
    return valid

In [ ]:
num_reads = 1000

response = sampler.sample(bqm_spin, num_reads=num_reads,
                          annealing_time=20,
                          num_spin_reversal_transforms=2,
                          postprocess='optimization',
                          answer_mode='raw')

tused = response.info['timing']['qpu_access_time']
print(f"QPU time used: {tused/1e3:.0f} ms or {tused/num_reads/1e3:.3f} ms/read")

dat = response.record.energy
# plt.hist(dat)
print(f"mean: {np.mean(dat):.2f}, sd: {np.std(dat):.2f}")
# plt.show()

num_cores = 12
par_res = Parallel(n_jobs=num_cores)(delayed(separate_subsamples)(sample, occurrences, response.vartype) for sample, occurrences in response.data(['sample', 'num_occurrences']))
subsamp_lst, occurre_lst = zip(*par_res)

all_subsamp = np.array(sum(subsamp_lst, []))
all_occurre = np.array(sum(occurre_lst, []))

all_sbsvald = np.array(Parallel(n_jobs=num_cores)(delayed(check_valid_bqsample)(smpl) for smpl in all_subsamp))
all_sbsener = np.array(Parallel(n_jobs=num_cores)(delayed(bqm_00.energy)(smpl) for smpl in all_subsamp))
all_sbsstat = np.array([[smpl[k] for k in bqm_00.variables] for smpl in all_subsamp])

sbsorder = np.argsort(all_sbsener)
all_subsamp = all_subsamp[sbsorder]
all_occurre = all_occurre[sbsorder]
all_sbsvald = all_sbsvald[sbsorder]
all_sbsener = all_sbsener[sbsorder]
all_sbsstat = all_sbsstat[sbsorder]
updated_tab = False

print(all_sbsener[:9].round(4))
print(all_sbsener[all_sbsvald][:9].round(4))

In [ ]:
viz_short_smpl(all_subsamp[all_sbsvald][0], prot_seq, q, energy_func)

In [ ]:
plt.plot(all_sbsener[all_sbsvald])
plt.show()

In [ ]:
plt.plot(all_sbsener[all_sbsvald&(all_sbsener<0)])
plt.show()

In [ ]:
plt.hist(all_sbsener[all_sbsvald], [-0.05 + 0.005*i for i in range(60)])
plt.show()

In [ ]:
tab_sbsstat, cnt_sbsstat = np.unique(all_sbsstat, axis=0, return_counts=True)
tab_subsamp = np.array([dict(zip(bqm_00.variables, stat)) for stat in tab_sbsstat])
tab_sbsvald = np.array(Parallel(n_jobs=num_cores)(delayed(check_valid_bqsample)(smpl) for smpl in tab_subsamp))
tab_sbsener = np.array(Parallel(n_jobs=num_cores)(delayed(bqm_00.energy)(smpl) for smpl in tab_subsamp))

tab_sbsordr = np.argsort(tab_sbsener)
tab_subsamp = tab_subsamp[tab_sbsordr]
tab_sbsvald = tab_sbsvald[tab_sbsordr]
tab_sbsener = tab_sbsener[tab_sbsordr]
tab_sbsstat = tab_sbsstat[tab_sbsordr]
cnt_sbsstat = cnt_sbsstat[tab_sbsordr]

In [ ]:
qq = [qi for qi in q if qi in bqm_00.variables]
ee = energy_expr.subs({q[0]: 0, q[1]: 1, q[2]: 0})
ee_func = lambdify(qq, ee)
tt = [t.subs({q[0]: 0, q[1]: 1, q[2]: 0}) for t in ene_terms_expr]
hb_f, ho_f, hi_f = [lambdify(qq, h) for h in tt] 

In [ ]:
df = pd.DataFrame({
    "state":[''.join([str(v) for k, v in s.items() if type(k) != str]) for s in tab_subsamp], 
    "isvalid": tab_sbsvald,
    "energy": tab_sbsener,
    "Hb": [hb_f(**{str(k): v for k, v in sample.items() if type(k) != str}) for sample in tab_subsamp],
    "Ho": [ho_f(**{str(k): v for k, v in sample.items() if type(k) != str}) for sample in tab_subsamp],
    "Hi": [hi_f(**{str(k): v for k, v in sample.items() if type(k) != str}) for sample in tab_subsamp],
    "count": cnt_sbsstat})

In [ ]:
df[(df.isvalid & (df.energy < 0))]